# Memoria explicativa

### Pisos turisticos Euskadi COMPLETO


Este script carga y limpia el archivo csv de viviendas turísticas en Euskadi facilitado por el INE, se puede descargar desde esta fuente: Fuente: https://www.ine.es/jaxiT3/Datos.htm?t=39364. Primero, se lee el archivo CSV (líneas 2 y 4), aunque uno de los read_csv es redundante. Se filtran solo las filas correspondientes a "Viviendas turísticas" y se limpia la columna "Total" aplicando la función limpiar_numero, que elimina puntos de miles, espacios y símbolos no numéricos para convertir los valores en enteros o asignar None si no es posible. A continuación, se extrae el año de la columna "Periodo" y se filtran los datos provinciales. De las provincias se extrae el nombre limpio y se renombran las tres provincias vascas para unificarlas (Araba, Bizkaia y Gipuzkoa). Luego se agrupan los datos por provincia y año para obtener el valor máximo de viviendas turísticas registrado en cada caso. De forma paralela, se extraen también los datos de la Comunidad Autónoma del País Vasco (CAE), agrupando por año y obteniendo igualmente el máximo de viviendas turísticas. Ambos conjuntos de datos (provincial y autonómico) se combinan en un solo DataFrame. Finalmente, se crea una tabla pivote que organiza los valores por territorio (filas) y año (columnas), ordenados de forma descendente, y se guarda el resultado limpio en un nuevo archivo CSV.



In [3]:
import pandas as pd

ine_pisos_tur = pd.read_csv('./data/viviendas_turisticas_INE.csv', encoding='latin-1')


df = pd.read_csv("./data/viviendas_turisticas_INE.csv", sep=';', encoding='latin1')

df = df[df["Viviendas y plazas"] == "Viviendas turísticas"]

# Función para limpiar los valores numéricos que están mal escritos con . los miles
def limpiar_numero(valor):
    if pd.isna(valor):
        return None
    valor_str = str(valor).strip()
    valor_limpio = valor_str.replace('.', '')
    valor_limpio = valor_limpio.replace(' ', '')
    if valor_limpio == '' or valor_limpio == '-' or valor_limpio == '..':
        return None
    try:
        return int(valor_limpio)
    except ValueError:
        print(f"No se pudo convertir: '{valor}' -> '{valor_limpio}'")
        return None

df["Viviendas"] = df["Total"].apply(limpiar_numero)

df["Año"] = df["Periodo"].str[:4]

prov_df = df[df["Provincias"].notna()].copy()
prov_df["Entidad"] = prov_df["Provincias"].str.extract(r'\d{2}\s+(.*)')
prov_df["Entidad"] = prov_df["Entidad"].replace({
    "Araba/Álava": "Araba",
    "Bizkaia": "Bizkaia",
    "Gipuzkoa": "Gipuzkoa"
})

prov_max = prov_df.groupby(["Entidad", "Año"])["Viviendas"].max().reset_index()

cae_df = df[(df["Provincias"].isna()) & (df["Comunidades y Ciudades Autónomas"] == "16 País Vasco")]
cae_max = cae_df.groupby("Año")["Viviendas"].max().reset_index()
cae_max["Entidad"] = "CAE"

final_df = pd.concat([prov_max, cae_max], ignore_index=True)

df_pivot = final_df.pivot(index="Entidad", columns="Año", values="Viviendas")

df_pivot = df_pivot[sorted(df_pivot.columns, reverse=True)]

df_final = df_pivot.reset_index()
df_final.columns.name = None
df_final = df_final.rename(columns={"Entidad": "Territorio"})

df_final.to_csv("./data/viviendas_turisticas.csv", sep=';', index=False)

df_final.head()

,Territorio,2024,2023,2022,2021,2020
0,Araba,4070,3320,3250,3280,3320
1,Bizkaia,2947,2545,2074,1928,2025
2,CAE,5828,5121,4693,4354,4598
3,Gipuzkoa,2474,2256,2294,2098,2241


### Precio alquiler Euskadi - Territorios históricos

Este script procesa un archivo csv con datos de precios de alquiler por trimestre en Euskadi, se puede descargar desde esta Fuente: https://www.eustat.eus/elementos/ele0017700/renta-mensual-media-de-los-contratos-de-alquiler-libre-de-vivienda-habitual-colectiva-contrato-de-vivienda-habitual-por-territorio-historico-y-tamano-del-municipio-segun-trimestre-de-inicio-del-contrato-eurosmes/tbl0017789_c.html. Lee el archivo saltando las primeras filas, filtra solo los registros de la CAE y sus tres provincias, y normaliza los nombres de los territorios. Luego elimina columnas vacías, convierte los valores numéricos al formato correcto (reemplazando comas por puntos) y calcula la media anual del precio de alquiler por territorio a partir de los valores trimestrales. Finalmente, genera un nuevo DataFrame con estos promedios anuales y lo guarda como un nuevo archivo csv.



In [4]:
import pandas as pd

archivo = "./data/precio_alquiler.csv"

df = pd.read_csv(archivo, sep=';', skiprows=5, header=None, encoding='latin1')

territorios = ['C.A. de Euskadi', '   Araba/Álava', '   Bizkaia', '   Gipuzkoa']
df = df[df[0].isin(territorios)].reset_index(drop=True)

df[0] = df[0].str.strip().replace({
    'C.A. de Euskadi': 'CAE',
    'Araba/Álava': 'Araba',
    'Bizkaia': 'Bizkaia',
    'Gipuzkoa': 'Gipuzkoa'
})

df = df.dropna(axis=1, how='all')

data_only = df.iloc[:, 1:].applymap(lambda x: str(x).replace(',', '.')).astype(float)

años = list(range(2016, 2025))
n_trimestres = 4

columnas_anuales = {}
for i, año in enumerate(años):
    inicio = i * n_trimestres
    fin = inicio + n_trimestres
    columnas_anuales[año] = data_only.iloc[:, inicio:fin].mean(axis=1)

df_resultado = pd.DataFrame(columnas_anuales)
df_resultado.insert(0, 'Territorio', df[0])

df_resultado.to_csv('./data/precio_alquiler_.csv')

df_resultado

df = pd.read_csv("./data/precio_alquiler_.csv", sep=',', encoding="latin1")
df = df.drop(columns=df.columns[0])

df.to_csv('./data/precio_alquiler_final.csv', index=False)

df.head()

C:\Users\Eduardo\AppData\Local\Temp\ipykernel_12624\3307703703.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_only = df.iloc[:, 1:].applymap(lambda x: str(x).replace(',', '.')).astype(float)


,Territorio,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,CAE,628.611352,638.736288,667.306500,688.964002,702.334793,709.413165,735.088313,776.313490,795.721470
1,Araba,544.793245,559.165458,592.394645,621.516687,634.338480,644.832238,678.004437,718.343090,723.885465
2,Bizkaia,644.169768,655.916545,690.234163,713.666588,729.841342,737.460952,762.392385,809.144362,828.982632
3,Gipuzkoa,642.550235,650.212623,671.556517,690.769225,700.095973,706.987067,731.464312,770.008853,792.404865


### Precio alquiler Euskadi - Territorios históricos
Este script extrae y transforma datos trimestrales de precios de alquiler en Euskadi, lo puedes descargar desde esta Fuente: https://www.eustat.eus/elementos/ele0017700/renta-mensual-media-de-los-contratos-de-alquiler-libre-de-vivienda-habitual-colectiva-contrato-de-vivienda-habitual-por-territorio-historico-y-tamano-del-municipio-segun-trimestre-de-inicio-del-contrato-eurosmes/tbl0017789_c.html. Lo que hace el script:
Carga el archivo CSV ignorando las primeras filas, selecciona solo las filas de la CAE y sus tres provincias, y estandariza los nombres de los territorios. A continuación, limpia los datos eliminando columnas vacías y convierte los valores numéricos al formato decimal correcto. Luego calcula el precio medio anual de alquiler para cada territorio a partir de los trimestres correspondientes (2016–2024). Finalmente, construye un nuevo DataFrame con estos promedios anuales y lo guarda en un archivo CSV.

In [6]:
import pandas as pd

archivo = "./data/precio_alquiler.csv"

df = pd.read_csv(archivo, sep=';', skiprows=5, header=None, encoding='latin1')

territorios = ['C.A. de Euskadi', '   Araba/Álava', '   Bizkaia', '   Gipuzkoa']
df = df[df[0].isin(territorios)].reset_index(drop=True)

df[0] = df[0].str.strip().replace({
    'C.A. de Euskadi': 'CAE',
    'Araba/Álava': 'Araba',
    'Bizkaia': 'Bizkaia',
    'Gipuzkoa': 'Gipuzkoa'
})

df = df.dropna(axis=1, how='all')

data_only = df.iloc[:, 1:].applymap(lambda x: str(x).replace(',', '.')).astype(float)

años = list(range(2016, 2025))
n_trimestres = 4

columnas_anuales = {}
for i, año in enumerate(años):
    inicio = i * n_trimestres
    fin = inicio + n_trimestres
    columnas_anuales[año] = data_only.iloc[:, inicio:fin].mean(axis=1)

df_resultado = pd.DataFrame(columnas_anuales)
df_resultado.insert(0, 'Territorio', df[0])

df_resultado.to_csv('./data/precio_alquiler_.csv')

df_resultado

df = pd.read_csv("./data/precio_alquiler_.csv", sep=',', encoding="latin1")
df = df.drop(columns=df.columns[0])

df.to_csv('./data/precio_alquiler_final.csv', index=False)

df.head()


C:\Users\Eduardo\AppData\Local\Temp\ipykernel_12624\1498483172.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_only = df.iloc[:, 1:].applymap(lambda x: str(x).replace(',', '.')).astype(float)


,Territorio,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,CAE,628.611352,638.736288,667.306500,688.964002,702.334793,709.413165,735.088313,776.313490,795.721470
1,Araba,544.793245,559.165458,592.394645,621.516687,634.338480,644.832238,678.004437,718.343090,723.885465
2,Bizkaia,644.169768,655.916545,690.234163,713.666588,729.841342,737.460952,762.392385,809.144362,828.982632
3,Gipuzkoa,642.550235,650.212623,671.556517,690.769225,700.095973,706.987067,731.464312,770.008853,792.404865


## Precio compraventa libre - COMPLETO
Esta función carga un csv con datos de precios de compraventa de vivienda en Euskadi, disponible para su descarga desde este enlace: https://www.eustat.eus/elementos/ele0013600/precio-medio-total-de-las-compraventas-de-vivienda-libre-de-la-ca-de-euskadi-por-trimestre-de-inscripcion-en-el-registro-segun-territorio-historico-y-tipo-de-vivienda-/tbl0013664_c.html, eliminando las columnas vacías y asignando nombres claros a las columnas. Convierte a formato numérico todas las columnas salvo la del trimestre, gestionando posibles errores. Si se proporciona una ruta de salida, guarda el DataFrame limpio como nuevo archivo csv. Devuelve el DataFrame procesado para su uso posterior que a su vez convertimos a csv para respaldo.



In [7]:
import pandas as pd

def limpiar_csv_compraventa(ruta_csv, ruta_salida=None):
    
    df = pd.read_csv(ruta_csv, sep=';', skiprows=6, encoding='latin1', header=None)

    df.dropna(axis=1, how='all', inplace=True)

    columnas = ['Trimestre', 
                'Euskadi_Total', 'Euskadi_Nueva', 'Euskadi_Usada',
                'Álava_Total', 'Álava_Nueva', 'Álava_Usada',
                'Bizkaia_Total', 'Bizkaia_Nueva', 'Bizkaia_Usada',
                'Gipuzkoa_Total', 'Gipuzkoa_Nueva', 'Gipuzkoa_Usada']
    df.columns = columnas

    for col in columnas[1:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')

        df.to_csv(ruta_salida, index=False)

    return df

limpiar_csv_compraventa('./data/precio.csv','./data/precio_cv_limpio.csv')
precio_cv_limpio = pd.read_csv('./data/precio_cv_limpio.csv')
precio_cv_limpio.head()

,Trimestre,Euskadi_Total,Euskadi_Nueva,Euskadi_Usada,Álava_Total,Álava_Nueva,Álava_Usada,Bizkaia_Total,Bizkaia_Nueva,Bizkaia_Usada,Gipuzkoa_Total,Gipuzkoa_Nueva,Gipuzkoa_Usada
0,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,I,267114.0,297774.0,260271.0,224678.0,230595.0,222308.0,247311.0,279833.0,239124.0,315334.0,408446.0,303217.0
2,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IV,266692.0,285388.0,262782.0,236454.0,230058.0,238988.0,249356.0,262106.0,246462.0,303779.0,390071.0,292504.0
4,III,254463.0,285509.0,248547.0,226078.0,242534.0,218616.0,237741.0,281675.0,228817.0,292993.0,371597.0,286166.0


Faltaría la función para cambiar los NaN de los años por la media de los trimestres y borrar los datos de los trimestres, que es lo que hace el siguiente script. Selecciona solo las columnas que terminan en "_Total" junto con la columna "Trimestre", y crea un nuevo DataFrame con esos datos. Luego renombra la columna "Trimestre" como "Año" porque será la forma de relacionar más tarde los diferentes csv.


In [8]:
import pandas as pd

def precio_año_media(df):
    df = df.copy()
    filas_a_conservar = []
    
    i = 0
    while i < len(df) - 4:
        fila = df.iloc[i]
        if str(fila['Trimestre']).isdigit() and pd.isna(fila[1:]).all():
            # Fila con año y NaNs: calcular media de los 4 trimestres siguientes
            medias = df.iloc[i+1:i+5].mean(numeric_only=True)
            fila_actualizada = fila.copy()
            fila_actualizada[1:] = medias.values
            filas_a_conservar.append(fila_actualizada)
            i += 5  # Saltar año + 4 trimestres
        else:
            i += 1  

    df_anual = pd.DataFrame(filas_a_conservar).reset_index(drop=True)
    return df_anual

df_venta_anual = precio_año_media(precio_cv_limpio)

df_venta_anual.head()

precio_venta_antestrasp = df_venta_anual.to_csv('./data/precio_venta_antestrasp.csv')

df = pd.read_csv('./data/precio_venta_antestrasp.csv', index_col=0)

cols_totales = [col for col in df.columns if col.endswith('_Total')]
cols_seleccionadas = ['Trimestre'] + cols_totales

df_totales = df[cols_seleccionadas].copy()

df_totales = df_totales.rename(columns={'Trimestre': 'Año'})

print(df_totales.head())




    Año  Euskadi_Total  Álava_Total  Bizkaia_Total  Gipuzkoa_Total
0  2025  262756.333333    229070.00  244802.666667   304035.333333
1  2023  247215.000000    221106.75  234635.750000   276812.750000
2  2022  243582.250000    207139.25  236409.500000   271810.250000
3  2021  246830.750000    202848.75  241695.500000   271371.750000
4  2020  241276.000000    198174.50  237037.250000   265791.250000


Por último extraemos las columnas totales por territorio junto a la columna de trimestres. Renombramos "Trimestre" como "Año", transponemos el DataFrame para dejar los territorios como filas y los años como columnas, y guardamos los nombres de los territorios históricos (Araba, Bizkaia, Gipuzkoa) y la CAE, que será el modelo que usaremos a partir de ahora como referencia. Por último, guarda el DataFrame transformado en un nuevo archivo CSV final.



In [12]:
import pandas as pd

df = pd.read_csv('./data/precio_venta_antestrasp.csv', index_col=0)

#Para las medias seleccionamos las columnas
cols_totales = [col for col in df.columns if col.endswith('_Total')]
df_totales = df[['Trimestre'] + cols_totales]

df_totales = df_totales.rename(columns={'Trimestre': 'Año'})

df_transpuesto = df_totales.set_index('Año').T
df_transpuesto = df_transpuesto.rename(index={
    'Euskadi_Total': 'CAE',
    'Álava_Total': 'Araba',
    'Bizkaia_Total': 'Bizkaia',
    'Gipuzkoa_Total': 'Gipuzkoa'
})
df_transpuesto.columns = df_transpuesto.columns.astype(int)
df_transpuesto = df_transpuesto.rename(columns={2025: 2024})
df_transpuesto.index.name = 'Territorio'

df_transpuesto.to_csv('./data/precio_cv_final.csv')
df_transpuesto.head()



Año,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012
Territorio,,,,,,,,,,,,,
CAE,262756.333333,247215.00,243582.25,246830.75,241276.00,222329.25,212776.50,208043.75,208165.25,200153.5,204741.25,219858.5,234880.75
Araba,229070.000000,221106.75,207139.25,202848.75,198174.50,182892.00,165228.00,167638.25,157015.25,167763.0,169159.75,188964.5,196940.00
Bizkaia,244802.666667,234635.75,236409.50,241695.50,237037.25,217864.50,209359.25,202720.50,208024.25,200141.0,201503.50,219383.0,247993.25
Gipuzkoa,304035.333333,276812.75,271810.25,271371.75,265791.25,242861.50,235729.75,230885.50,224898.75,210553.5,225429.00,232727.5,240707.75
